In [1]:
import os
import utils
import dataloader
import torch
import torch.utils.data as Data
import torch.nn.init
import torchvision
from torch.autograd import Variable
import torchvision.utils as utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import time

%load_ext autoreload
%autoreload 2

HR_train, HR_valid, LR_train, LR_valid = dataloader.load_data()
#convert to torch Tensor
HR_train = torch.Tensor(HR_train)
HR_valid = torch.Tensor(HR_valid)
LR_train = torch.Tensor(LR_train)
LR_valid = torch.Tensor(LR_valid)
print("shape:",HR_train.shape,",",LR_train.shape)

trainset = Data.TensorDataset(data_tensor=LR_train, target_tensor=HR_train)
validset = Data.TensorDataset(data_tensor=LR_valid, target_tensor=HR_valid)
batch_num = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_num, shuffle=True, num_workers=10)
validloader = torch.utils.data.DataLoader(validset, batch_size=batch_num, shuffle=True, num_workers=10)
validloader2 = torch.utils.data.DataLoader(validset, batch_size=1, shuffle=True, num_workers=10)

1
shape: torch.Size([1]) , torch.Size([1])


TypeError: __init__() got an unexpected keyword argument 'data_tensor'

In [1]:
import numpy as np
import os
import scipy.misc
from models import *
%reload_ext autoreload
%autoreload 2

model = ESPCN(upscale_factor=3)
model.cuda()
print(model)

ESPCN(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(4, 4))
  (conv2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (pixel_shuffle): PixelShuffle(upscale_factor=3)
  (relu): ReLU()
  (tanh): Tanh()
)


In [ ]:
import numpy as np
import torch.optim as optim

MSE_loss = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4)

net.load_state_dict(torch.load('./checkpoints/ESPCN.pkl'))
start_time0 = time.time()
for epoch in range(2000):
    if epoch % 100 == 0:
        print('Epoch %d, runtime: %.4f' %(epoch + 1, time.time()-start_time))
    start_time = time.time()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        #print(i)
        LR, HR = data 
        HR = HR.type(torch.FloatTensor) 
        inputs, target = Variable(LR), Variable(HR)
        inputs = inputs.cuda()
        targets = targets.cuda()

        optimizer.zero_grad()
        
        outputs = net(inputs)
        #print("outputs:", outputs,"steering:",steering)
        
        loss = MSE_loss(outputs, targets)
        loss.backward()
        optimizer.step()
        
        loss += loss.data[0]
        if epoch % 100 == 0 and i % 50 == 49:   # print every 128 mini-batches
            print('[%d, %5d] loss : %.6f' % (epoch + 1, i+ 1, running_loss / batch_num))
            loss = 0.0
    
print('Finished training, runtime: %.4f' %(time.time()-start_time0))

torch.save(net.state_dict(), './checkpoints/ESPCN.pkl')